In [1]:
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import dask as d
import os
import json
from glob import glob

In [2]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs, zstacks
import fl_tissue_model_tools.preprocessing as prep

In [3]:
dirs = dev_config.get_dev_directories("../dev_paths.txt")

In [4]:
with open("../model_training/invasion_depth_best_hp.json", 'r') as fp:
    best_hp = json.load(fp)

In [5]:
best_hp

{'adam_beta_1': 0.9066466810625207,
 'adam_beta_2': 0.9947698608592066,
 'fine_tune_lr': 9.141354728081903e-05,
 'frozen_lr': 0.00022767552973325001,
 'last_resnet_layer': 'conv5_block1_out'}

In [6]:
zstacks_root_path = f"{dirs.data_dir}/invasion_data/demo"

In [7]:
model_training_path = f"{dirs.analysis_dir}/resnet50_invasion_model"
best_ensemble_training_path = f"{model_training_path}/best_ensemble"
inv_depth_output_path = f"{dirs.analysis_dir}/invasion_depth_output"

# Binary classification -> only need 1 output unit
cls_thresh = 0.5
n_outputs = 1
resnet_inp_shape = (128, 128, 3)
class_labels = {"no_invasion": 0, "invasion": 1}
last_resnet_layer = best_hp["last_resnet_layer"]
# Zstack index meaning (descending: 0 -> bottom of well, ascending: 0 -> top of well)
descending = True

In [8]:
data_prep.make_dir(inv_depth_output_path)

In [9]:
zstack_dirs = [fp.replace("\\", "/") for fp in glob(f"{zstacks_root_path}/*")]

In [10]:
zstack_dirs

['D:/fogg_lab_tissue_model_data/invasion_data/demo/03_17_2022_A1_Caski_paclitaxel_Ines_Cadena',
 'D:/fogg_lab_tissue_model_data/invasion_data/demo/08_31_2021_E5_SiHa_validation_Ines_Cadena',
 'D:/fogg_lab_tissue_model_data/invasion_data/demo/09_08_2021_C1_HEC_DOE_Ines_Cadena']

# Build models

In [11]:
n_models = 5

In [12]:
K.clear_session()
inv_depth_models = [
    models.build_ResNet50_TL(n_outputs, resnet_inp_shape, base_last_layer=last_resnet_layer, base_model_trainable=False) for _ in range(n_models)
]

In [13]:
for i, m in enumerate(inv_depth_models):
    # Weights don't load properly in trainable set to False for model
    # Set trainable to True, load weights, then set back to False
    m.trainable = True
    m.load_weights(f"../model_training/best_ensemble/best_finetune_weights_{i}.h5")
    # m.load_weights(f"{best_ensemble_training_path}/best_finetune_weights_{i}.h5")
    m.trainable = False

# Make predictions

In [14]:
for zstack_dir in zstack_dirs:
    ### Load data ###
    zpaths = zstacks.z_stack_paths_from_dir(zstack_dir, descending=descending)
    x = data_prep.prep_inv_depth_imgs(zpaths, resnet_inp_shape[:-1])
    # Convert to tensor before calling predict() to speed up execution
    x = tf.convert_to_tensor(x, dtype="float")
    
    ### Make predictions ###
    # Probability predictions of each model
    yhatp_m = np.array(
        d.compute([d.delayed(m.predict)(x).squeeze() for m in inv_depth_models])[0]
    ).T
    # Mean probability predictions (ensemble predictions)
    yhatp = np.mean(yhatp_m, axis=1, keepdims=True)
    # Threshold probability predictions
    yhat = (yhatp > cls_thresh).astype(np.int32)
    
    ### Save outputs ###
    zstack_id = zstack_dir.split("/")[-1]
    
    output_file = pd.DataFrame({"img_name": [zp.split("/")[-1] for zp in zpaths], "inv_prob": yhatp.squeeze(), "inv_label": yhat.squeeze()})
    output_file.to_csv(f"{inv_depth_output_path}/{zstack_id}.csv", index=False)